#4. Matrix Factorization(MF) 기반 추천

||메모리 기반 알고리즘|모델 기반 알고리즘|
|---|---|---|
|설명|메모리에 있는 데이터를 계산해서 <br> 추천하는 방식|데이터로부터 미리 모델을 구성 후, <br> 필요 시 추천하는 방식|
|특징|개별 사용자 데이터 집중|전체 사용자 패턴 집중|
|장점|원래 데이터에 충실하게 사용|대규모 데이터에 빠르게 반응|
|단점|대규모 데이터에 느리게 반응|모델 생성 과정 오래 걸림|
|예시|CF 기반 추천 알고리즘|MF 기반 추천 알고리즘, 딥러닝|

## 4.1. Matrix Factorization(MF) 방식의 원리
* Matrix Factorization : 행렬 요인화/분해
* 평가, 사용자, 아이템으로 구성된 하나의 행렬을, 두 개의 행렬로 분해
* R ≈ P × Q.T = R^ (예상 평점)
  * Rating matrix R - M×N 차원
  * User latent matrix 사용자와 사용자 잠재 요인 행렬 P - M×K 차원
  * Item latent matrix 아이템과 아이템 잠재 요인 행렬 Q - N×K 차원
* CF에서는 사용자와 아이템, 평점으로 이루어진 full-matrix R 이용

## 4.2. SGD(Stochastic Gradient Decent)를 사용한 MF 알고리즘
 * SGD를 사용해 MF의 P와 Q 행렬을 구하는 게 최종 목표  

### 4.2.1. MF 알고리즘 개념적 설명
> 1. 잠재 요인 개수 K 선택
> 2. P, Q 행렬 초기화  
>
> [ 반복 ]  
> > 3. 예측 평점 R_hat(= P×Q.T) 계산
> > 4. 실제 R과 R_hat 간 오차 계산 및 P, Q 수정 (오차 감소 위함)
> > → 가장 중요한 단계
> > 5. 기준 오차 도달 확인

* MF의 핵심 : P와 Q 잘 분해하기
  * 주어진 사용자와 아이템의 관계를 잘 설명할 수 있도록  

### 4.2.2. SGD : Stochastic Gradient Decent
* 예측 오차를 줄이기위한 P, Q 업데이트
  * 예측 오차 제곱의 편미분 값 사용
  * 학습률(learning rate) α 알파 활용
* Overfitting 과적합 방지
  * 정규화 고려
    * 정규화 항(Regulation term) 추가
    * 정교화 계수 β
  * 경향성 고려
    * 사용자와 아이템의 경향성 문제
      * 전체 평균 b
      * 전체 평균을 제거한 후 사용자 i의 평가 경향 bu[i]
        * 사용자 i 평균과 전체 평균의 차이
      * 전체 평균을 제거한 후 아이템 j의 평가 경향 bi[j]
        * 아이템 j의 평균과 전체 평균의 차이
    * CF에서는 사용자와 아이템 별로 평가 경향이 한 번에 계산되었는데  
    MF에서는 계산할 때마다 오차를 최소화하도록 bu[i]와 bi[j] 계속 업데이트




## 4.3. SGD를 사용한 MF 기본 알고리즘

In [1]:
import os
import pandas as pd
import numpy as np

base_src = "drive/MyDrive/RecoSys/python-recosys/Data"
u_data_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_data_src, sep = "\t", names = r_cols, encoding = "latin-1")
ratings = ratings[["user_id", "movie_id", "rating"]].astype(int)

In [2]:
class MF() :
  # hyper_params : 알파나 베타 값 등 딕셔너리로
  def __init__(self, ratings, hyper_params) :
    # 데이터프레임 형식으로 전달된 평점 넘파이 배열로 바꾸기
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    self.K = hyper_params["K"] # 잠재 요인 개수
    self.alpha = hyper_params["alpha"] # 학습률
    self.beta = hyper_params["beta"] # 정교화 계수
    self.iterations = hyper_params["iterations"] # SGD 얼만큼 반복
    self.verbose = hyper_params["verbose"] # 학습 과정 중간에 출력할 것인지 여부를 판단하는 플래그 변수

  # P와 Q를 이용해 RMSE를 계산하는 함수
  def rmse(self) :
    xs, ys = self.R.nonzero() # 0이 아닌 요소의 인덱스 반환
    self.predictions = [] # 나중에 prediction과 error를 담을 리스트 변수 초기화
    self.errors = []

    for x, y in zip(xs, ys) :
      prediction = self.get_prediction(x, y) # 사용자 x 아이템 y 에서 평점예측치 계산하는 함수
      self.predictions.append(prediction)
      # 실제값과 예측값의 차이를 오차값으로 설정
      self.errors.append(self.R[x, y] - prediction)
    self.predictions = np.array(self.predictions)
    self.errors = np.array(self.errors)

    return np.sqrt(np.mean(self.errors**2))

  # 학습 메소드
  def train(self) :
    # P와 Q 우선 난수값으로 초기화
    # mean을 지정하지 않으면 디폴트로 0
    # 표준 편차 sacle을 1/잠재변수개수 로 지정
    self.P = np.random.normal(scale = 1./self.K, size = (self.num_users, self.K))
    self.Q = np.random.normal(scale = 1./self.K, size = (self.num_items, self.K))

    # 사용자 평가 경향
    self.b_u = np.zeros(self.num_users)

    # 아이템
    self.b_d = np.zeros(self.num_items)

    # 평점의 전체 평균
    self.b = np.mean(self.R[self.R.nonzero()])

    # SGD를 적용할 대상 설정
    rows, columns = self.R.nonzero()
    # 평점의 인덱스와 평점을 리스트로 만들어서 저장
    self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

    # SGD가 한 번 실행될 때마다 RMSE가 얼마나 계산되는지 기록하는 리스트
    training_process = []
    for i in range(self.iterations) :
      # 다양한 시작점에서 SGD 적용
      # 데이터의 순서에 따라 모델의 학습 경로가 영향을 받을 수 있기 때문에, 데이터를 무작위로 섞는 것은 중요
      np.random.shuffle(self.samples)
      self.sgd()
      rmse = self.rmse()
      training_process.append((i+1, rmse))

      # SGD 학습 과정을 중간에 출력할 건지 여부
      if self.verbose :
        if (i+1) % 10 == 0 :
          print("Iteration : %d ; train RMSE = %.4f" %(i+1, rmse))
    return training_process

  # 평점 예측값 구하는 함수
  # 아이템 j에 대한 사용자 i의 평점 예측치
  def get_prediction(self, i, j) :
    # R_hat
    # 전체 평점 + 사용자 평가 경향 + 아이템에 대한 평가 경향 + (사용자 i의 요인 값과 아이템 j 요인의 행렬 연산)
    predriction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
    return predriction

  # 최적의 P, Q, B_U, B_D 구하기 위한 과정
  def sgd(self) :
    for i, j, r in self.samples :
      prediction = self.get_prediction(i, j)
      # 실제 평점과 비교해 오차 계산
      e = (r - prediction)

      # 사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      # 아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      # 행렬 P 계산 및 업데이트
      self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
      # 행렬 Q 계산 및 업데이트
      self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

In [3]:
R_temp = ratings.pivot(index = "user_id", columns = "movie_id", values = "rating").fillna(0)
hyper_params = {
    "K" : 30,
    "alpha" : 0.001,
    "beta" : 0.02,
    "iterations" : 100,
    "verbose" : True
    }
mf = MF(R_temp, hyper_params)
train_process = mf.train()

Iteration : 10 ; train RMSE = 0.9585
Iteration : 20 ; train RMSE = 0.9374
Iteration : 30 ; train RMSE = 0.9281
Iteration : 40 ; train RMSE = 0.9225
Iteration : 50 ; train RMSE = 0.9184
Iteration : 60 ; train RMSE = 0.9145
Iteration : 70 ; train RMSE = 0.9099
Iteration : 80 ; train RMSE = 0.9038
Iteration : 90 ; train RMSE = 0.8949
Iteration : 100 ; train RMSE = 0.8831


## 4.4. train/test 분리 MF 알고리즘

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

base_src = "drive/MyDrive/RecoSys/python-recosys/Data"
u_data_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_data_src, sep = "\t", names = r_cols, encoding = "latin-1")
ratings = ratings[["user_id", "movie_id", "rating"]].astype(int)

# train/test set 분리
## train_test_split을 사용했을 때는 stratify = y로 지정해 계층화 추출을 했었는데
## 집단 간 이질성이 크지 않은 경우 오히려 표본의 대표성을 저해할 수 있다
## suffle 방식은 완전 무작위
TRAIN_SIZE = 0.75

## 사용자 - 영화 - 평점
## random_state 난수 발생 시드, 시드가 같으면 같은 난수 시퀀스
ratings = shuffle(ratings, random_state = 2021)
## 데이터 몇 개 뽑을지
cutoff = int(TRAIN_SIZE * len(ratings))
## iloc : 인덱스로 지정
## loc : 레이블로 지정
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

class NEW_MF() :
  def __init__(self, ratings, hyper_params) :
    self.R = np.array(ratings)
    self.num_users, self.num_items = np.shape(self.R)
    # MF weight 조절을 위한 하이퍼 파라미터
    ## K : 잠재 요인(latent factor) 의 수
    self.K = hyper_params["K"]
    ## alpha : 학습률
    self.alpha = hyper_params["alpha"]
    ## beta : 정규화 계수
    self.beta = hyper_params["beta"]
    ## iterations : SGD 계산 시 반복 횟수
    self.iterations = hyper_params["iterations"]
    ## verbose : SGD 학습 과정을 중간중간 출력할 것인지 여부
    self.verbose = hyper_params["verbose"]

    # 전처리가 잘 이루어진 데이터 셋을 사용하기 때문에 사용자id나 아이템id가 연속된 숫자로 이루어져 있다
    # 실제 데이터는 그렇지 않을 수도
    # id와 seld.R의 인덱스가 일치하지 않을 수도
    ## 아이템 id
    item_id_index = []
    index_item_id = []
    ## 여기에서의 ratings는 full-matrix
    ## i는 인덱스 번호, one_id는 movie_id
    for i, one_id in enumerate(ratings) :
      item_id_index.append([one_id, i])
      index_item_id.append([i, one_id])
    self.item_id_index = dict(item_id_index)
    self.index_item_id = dict(index_item_id)

    ## 사용자 id
    user_id_index = []
    index_user_id = []
    for i, one_id in enumerate(ratings.T) :
      user_id_index.append([one_id, i])
      index_user_id.append([i, one_id])
    self.user_id_index = dict(user_id_index)
    self.index_user_id = dict(index_user_id)

  def rmse(self) :
    # self.R에서 평점이 있는 요소의 인덱스를 가져온다
    xs, ys = self.R.nonzero()
    # prediction과 error를 담을 리스트 변수 초기화
    self.predictions = []
    self.errors = []

    # 평점이 있는 요소(사용자 x, 아이템 y) 각각에 대해 아래 코드 실행
    for x, y in zip(xs, ys) :
      # 사용자 x, 아이템 y에 대해 평점 예측치를 get_prediction()으로 연산
      prediction = self.get_prediction(x, y)
      # 예측 리스트에 예측값 추가
      self.predictions.append(prediction)
      # 실제값 R과 예측값의 차이를 계산해서 오차값 리스트에 추가
      error = self.R[x, y] - prediction
      self.errors.append(error)
    # numpy array 형태로 변환
    self.predictions = np.array(self.predictions)
    self.errors = np.array(self.errors)

    # error 이용해서 rmse 도출
    return np.sqrt(np.mean(self.errors ** 2))

  def sgd(self) :
    for i, j, r in self.samples :
      # 사용자 i, 아이템 j에 대한 평점 예측치 계산
      prediction = self.get_prediction(i, j)
      # 실제 평점과 비교한 오차 계산
      e = (r - prediction)

      # 사용자 평가 경향 계산 및 업데이트
      self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
      # 아이템 평가 경향 계산 및 업데이트
      self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

      # 행렬 P 계산 및 업데이트
      self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
      # 행렬 Q 계산 및 업데이트
      self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

  def get_prediction(self, i, j) :
    # 사용자 i, 아이템 j에 대한 예측치
    # R_hat
    # 전체 평점 + 사용자 평가 경향 + 아이템에 대한 평가 경향 + (사용자 i의 요인 값과 아이템 j 요인의 행렬 연산)
    predriction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
    return predriction

  # Test Set 선정
  ## 분리된 테스트셋을 넘겨받아서 클래스 내부의 테스트셋을 만드는 함수
  def set_test(self, ratings_test) :
    test_set = []
    for i in range(len(ratings_test)) :
      # 실제 테스트 셋에 있는 사용자와 아이템 인덱스, 평점 받아오기
      x = self.user_id_index[ratings_test.iloc[i, 0]]
      y = self.item_id_index [ratings_test.iloc[i, 1]]
      z = ratings_test.iloc[i, 2]

      # 테스트 셋 만들기
      test_set.append([x,y,z])

      # full-matrix에서 테스트 셋이 된 데이터는 지우는 작업
      self.R[x,y] = 0

    self.test_set = test_set
    return test_set

  # 테스트 셋 rmse 계산
  def test_rmse(self) :
    error = 0
    for one_set in self.test_set :
      predicted = self.get_prediction(one_set[0], one_set[1])
      error += pow(one_set[2] - predicted, 2)
    return np.sqrt(error/len(self.test_set))

  # 학습하면서 테스트 셋의 정확도 계산
  def test(self) :
    self.P = np.random.normal(scale = 1./self.K, size = (self.num_users, self.K))
    self.Q = np.random.normal(scale = 1./self.K, size = (self.num_items, self.K))
    self.b_u = np.zeros(self.num_users)
    self.b_d = np.zeros(self.num_items)
    self.b = np.mean(self.R[self.R.nonzero()])
    # 아까 테스트 셋 생성할 때 0으로 바꾸어줘서 지금 여기에서 트레인 셋들의 평균만 얻을 수 있음

    # 트레인 셋에 대해 데이터 구성
    rows, columns = self.R.nonzero()
    self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

    training_process = []
    for i in range(self.iterations) :
      np.random.shuffle(self.samples)
      self.sgd()
      rmse1 = self.rmse()
      rmse2 = self.test_rmse()
      training_process.append((i+1, rmse1, rmse2))
      if self.verbose :
        if (i+1) % 10 == 0 :
          print("Iteration : %d ; Train RMSE = %.4f ; Test RMSE = %.4f" %(i+1, rmse1, rmse2))

    return training_process

  # 주어진 사용자와 아이템에 대해 예측치 계산
  def get_one_prediction(self, user_id, item_id) :
    return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])

  def full_prediction(self) :
    return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)
    # np.newaxis 행렬 연산하기 위해

In [2]:
R_temp = ratings.pivot(index = "user_id", columns = "movie_id", values = "rating").fillna(0)
hyper_params = {
    "K" : 30,
    "alpha" : 0.001,
    "beta" : 0.02,
    "iterations" : 100,
    "verbose" : True
}
mf = NEW_MF(R_temp, hyper_params)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration : 10 ; Train RMSE = 0.9666 ; Test RMSE = 0.9807
Iteration : 20 ; Train RMSE = 0.9412 ; Test RMSE = 0.9622
Iteration : 30 ; Train RMSE = 0.9297 ; Test RMSE = 0.9551
Iteration : 40 ; Train RMSE = 0.9228 ; Test RMSE = 0.9514
Iteration : 50 ; Train RMSE = 0.9179 ; Test RMSE = 0.9491
Iteration : 60 ; Train RMSE = 0.9138 ; Test RMSE = 0.9476
Iteration : 70 ; Train RMSE = 0.9099 ; Test RMSE = 0.9463
Iteration : 80 ; Train RMSE = 0.9057 ; Test RMSE = 0.9451
Iteration : 90 ; Train RMSE = 0.9004 ; Test RMSE = 0.9436
Iteration : 100 ; Train RMSE = 0.8936 ; Test RMSE = 0.9417


In [3]:
print(mf.full_prediction())

[[3.87980178 3.34338145 2.96055887 ... 3.34092972 3.48263425 3.45461261]
 [3.8244984  3.27396861 2.91573086 ... 3.25672117 3.3663734  3.35885352]
 [3.43717231 2.9087714  2.51649303 ... 2.8598944  2.98732412 2.95053395]
 ...
 [4.14928322 3.61034908 3.23668372 ... 3.58728544 3.70442883 3.68045791]
 [4.30757931 3.78933879 3.41166885 ... 3.74856465 3.87458828 3.85804816]
 [3.86248011 3.33107724 2.91803774 ... 3.3005338  3.38639646 3.3819701 ]]


In [4]:
print(mf.get_one_prediction(1, 2))
# 사용자 1이 아이템 2에 대한 예측

3.3433814480640756


## 4.5. MF 최적 파라미터 찾기
> 1. 대략적인 최적의 K 위치 찾기
> 2. 대략적 K 주변 탐색으로, 최적 K 찾기
> 3. 주어진 K 통해 최적의 iterations 선

* K외에도 다양한 파라미터 alpha, beta 등등에 대해
  * 개개별 최적의 파라미터를 구했다 할 지라도
  * 이들의 단순 조합이 최고의 결과를 낸다는 보장은 없음
  

In [5]:
# 최적의 k 찾기

## 성능에 대한 결과값 담기
results = []
## 어떤 K일 때 어떤 결과가 나왔는지 나타내기 위함
index = []

R_temp = ratings.pivot(index = "user_id", columns = "movie_id", values = "rating").fillna(0)

for K in range(50, 261, 20) :
  print(f"K : {K}")
  hyper_params = {
      "K" : K,
      "alpha" : 0.001,
      "beta" : 0.02,
      "iterations" : 300,
      "verbose" : True
  }
  mf = NEW_MF(R_temp, hyper_params)
  test_set = mf.set_test(ratings_test)
  result = mf.test()
  index.append(K)
  results.append(result)

K : 50
Iteration : 10 ; Train RMSE = 0.9669 ; Test RMSE = 0.9807
Iteration : 20 ; Train RMSE = 0.9417 ; Test RMSE = 0.9622
Iteration : 30 ; Train RMSE = 0.9305 ; Test RMSE = 0.9552
Iteration : 40 ; Train RMSE = 0.9240 ; Test RMSE = 0.9515
Iteration : 50 ; Train RMSE = 0.9195 ; Test RMSE = 0.9493
Iteration : 60 ; Train RMSE = 0.9161 ; Test RMSE = 0.9479
Iteration : 70 ; Train RMSE = 0.9132 ; Test RMSE = 0.9469
Iteration : 80 ; Train RMSE = 0.9102 ; Test RMSE = 0.9460
Iteration : 90 ; Train RMSE = 0.9069 ; Test RMSE = 0.9452
Iteration : 100 ; Train RMSE = 0.9027 ; Test RMSE = 0.9441
Iteration : 110 ; Train RMSE = 0.8970 ; Test RMSE = 0.9426
Iteration : 120 ; Train RMSE = 0.8893 ; Test RMSE = 0.9405
Iteration : 130 ; Train RMSE = 0.8791 ; Test RMSE = 0.9376
Iteration : 140 ; Train RMSE = 0.8664 ; Test RMSE = 0.9342
Iteration : 150 ; Train RMSE = 0.8516 ; Test RMSE = 0.9305
Iteration : 160 ; Train RMSE = 0.8351 ; Test RMSE = 0.9272
Iteration : 170 ; Train RMSE = 0.8174 ; Test RMSE = 0.9244

In [6]:
summary = []
for i in range (len(result)) :
  RMSE = []
  for result in results[i] :
    # result에서 0번째는 그냥 인덱스, 1번째는 train에 대한 RMSE, 2번째는 test에 대한 RMSE
    RMSE.append(result[2])
  min = np.min(RMSE)
  # 몇 번째의 iteration이 최소의 RMSE를 갖게 되는지
  j = RMSE.index(min)
  # summary에 K값, 최적의 RMSE iteration 값, 최소의 RMSE 값
  summary.append([index[i], j+1, RMSE[j]])

IndexError: ignored

In [ ]:
# 그래프를 보면 K가 150 근처에서 최적값

In [ ]:
summary

## 4.6. MF와 SVD
### 4.6.1. MF
* 원래의 행렬을 두 개의 행렬로 나눈다
* null 값을 0으로 표현하긴 하지만,  
  P와 Q에 대해 연산할 때 0에 대한 값은 빼고 계산하기 때문에   
  사실 null 값은 제외하고 계산하는 구조
* 원래 행렬에 null 값이 있다하더라도, P와 Q 행렬은 null 없이 계산하기 때문에  
  학습이 끝나고 나면 원래 행렬의 null 값에 대해서도 상당히 정확하게 예측할 수 있음

### 4.6.2. SVD
* 원래의 행렬을 세 개의 행렬로 나누어 학습한 뒤 다시 원래의 행렬로 재현
* 원래 행렬의 null값을 허용하지 않기 때문에   
  원래 행렬에 없던 값을 예측하는 데에는 문제가 있다
* null값을 0으로 대체하면   
  0 또한 하나의 값으로 인식하고 이 0값에 대해 예측하기 위해 노력하기 때문에  
  0이 아니라 0에 가까운 값으로 예측